In [ ]:
import json
from pathlib import Path
import pickle

import numpy as np 

from astropy.table import Table 

#from try_json_editing_rev1 import whole_thing, generate_json_file_name, exotic_arguments, get_values_from_widget
from stellarphot.transit_fitting.gui import generate_json_file_name, exotic_arguments, get_values_from_widget, exotic_settings_widget, populate_TOI_boxes
from stellarphot.settings.fits_opener import FitsOpener
from stellarphot import PhotometryData
from stellarphot.io import TOI
from stellarphot.gui_tools.photometry_widget_functions import TessAnalysisInputControls

# Instructions and Overview

This notebook will do a different kind of fit to your exoplanet data. It takes longer to run than the fitting we've done so far, which is why we didn't begin with this type of fit.

### 0. Get some data

+ Select photometry file with relative flux
+ Select passband
+ Select TESS info file

In [ ]:
taic = TessAnalysisInputControls()
taic

In [ ]:
# 👉 File with photometry, including flux
photometry_file = taic.photometry_data_file
inp_photometry = taic.phot_data

# 👉 File with exoplanet info in
tess_info_output_file = taic.tic_info_file
tess_info = TOI.model_validate_json(tess_info_output_file.read_text())

### Get just the target star and some information about it

In [ ]:
if np.isnan(inp_photometry['bjd']).all():
    inp_photometry.add_bjd_col()

In [ ]:
photometry = inp_photometry.lightcurve_for(1, flux_column="relative_flux", passband=taic.passband).remove_nans()

In [ ]:
settings = exotic_settings_widget()

exotic_data_file = 'data_for_exotic.csv'

In [ ]:

out_data = photometry['bjd', 'relative_flux', 'relative_flux_error', 'airmass']
out_data.write(exotic_data_file, overwrite=True)

In [ ]:
settings.data_file_widget['candidate'].value = exotic_data_file
settings.data_file_widget['known'].value = exotic_data_file
populate_TOI_boxes(tess_info, settings.value_widget)

## YOU MAY NEED DATA FROM HERE:  https://exofop.ipac.caltech.edu/tess/


In [ ]:
settings

In [ ]:
settings_name = generate_json_file_name(settings)
contents = get_values_from_widget(settings)

In [ ]:
Path(contents['user_info']['Directory to Save Plots']).mkdir(exist_ok=True)

In [ ]:
with open(settings_name, 'w') as f:
    json.dump(contents, f)

In [ ]:
print('COPY AND PASTE THE LINE BELOW INTO AN EMPTY CELL TO RUN EXOTIC (include the exclamation point at the beginning):\n\n')
print(f'!exotic {" ".join(exotic_arguments[settings.planet_type.value])} {settings_name}')
print("\n\n")

In [ ]:
!exotic --override --pre TIC_370981403-2024-10-18-SG.json